![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Multimodales RAG
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
    ('markitdown[all]', 'markitdown'),
    'langchain_chroma',
])

In [ ]:
#@title 📂 Dokumente und Bilder kopieren { display-mode: "form" }
!rm -rf files
!mkdir files

# --- Texte
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_1.txt -o files/biografien_1.txt
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_2.md -o files/biografien_2.md
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_3.pdf -o files/biografien_3.pdf
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_4.docx -o files/biografien_4.docx
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/roboter.txt -o files/roboter.txt

# --- Bilder
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/a_retro-futuristic_robot_dall_e.jpg -o files/a_retro-futuristic_robot_dall_e.jpg
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/hedra_cyborg.png -o files/hedra_cyborg.png
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/apfel.jpg -o files/apfel.jpg
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/zwei_roboter.png -o zwei_roboter.png

# --- Python-Modul
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/01%20ipynb/multimodal_rag_modul.py -o multimodal_rag_modul.py

# 1 | Setup multimodal RAG
---

Modul für ein **funktionales multimodales RAG-System**, das Text- und Bilddokumente in einer einheitlichen Vektordatenbank verwaltet und durchsucht.

**Modalitätsrichtungen**

| Eingabe (Query) | Ausgabe (Antwort) | Beispiel / Beschreibung | Status |
|-----------------|-------------------|-------------------------|--------|
| **Text → Text** | Textbasierte Frage führt zu Textantwort | Klassisches RAG-System (z.B. Chatbot, Q&A) | ✅ |
| **Text → Bild** | Textanfrage findet relevante Bilder | "Zeige mir Roboter-Bilder" | ✅ |
| **Bild → Text** | Bildanalyse oder Captioning | "Was ist auf diesem Foto zu sehen?" | ✅ |
| **Bild → Bild** | Bildretrieval oder visuelle Transformation | "Finde ähnliche Bilder" | ✅ |
| **Text + Bild → Text** | Kombination zur Textgenerierung | "Welche Informationen enthält dieses Diagramm?" | ❌ |
| **Text + Bild → Bild** | Bedingte Bildgenerierung | "Mach aus diesem Bild eine Winterversion" | ❌ |

**Hauptvorteile**

1. **Funktionale Architektur**: Klare Trennung von Konfiguration, Komponenten und Funktionen
2. **Einheitliche Datenbank**: ChromaDB mit separaten Collections für Text und Bilder
3. **Hybride Suche**: Text-Embeddings (OpenAI) + Bild-Embeddings (CLIP)
4. **Flexible Konfiguration**: Alle Parameter über RAGConfig anpassbar
5. **Produktionsreif**: Mit Error-Handling, Duplikats-Prüfung und Status-Monitoring

<p><font color='black' size="5">
Zentrale Komponenten
</font></p>

**RAGConfig**
- Zentrale Konfigurationsklasse
- Anpassbare Parameter (chunk_size, models, thresholds)

**RAGComponents**
- Container für alle System-Komponenten
- Text-Embeddings, CLIP-Model, LLMs, Collections

**Hauptfunktionen**
- `init_rag_system_enhanced()`: System-Initialisierung
- `process_directory()`: Bulk-Import von Dateien
- `add_text_document()`: Einzelnes Dokument hinzufügen
- `add_image_with_description()`: Bild mit Auto-Beschreibung
- `search_texts()`: Text-Suche inkl. Bildbeschreibungen
- `search_images()`: CLIP-basierte Bildsuche
- `search_similar_images()`: Bild→Bild Ähnlichkeitssuche
- `search_text_by_image()`: Bild→Text Suche (NEU)
- `multimodal_search()`: Erweiterte multimodale Suche

In [ ]:
# Import des Moduls
from multimodal_rag_modul import init_rag_system_enhanced, process_directory, multimodal_search, search_text_by_image, search_similar_images

# 2 | RAG initialisieren
---

In [ ]:
# Initialisierung
rag = init_rag_system_enhanced()

# 3 | Bilder/Texte in RAG übernehmen
---

In [ ]:
# Dokumente/Bilder laden & verarbeiten
process_directory(rag, './files', auto_describe_images=True)

# 4 | Suche Text -> Text/Bild
---

In [ ]:
result = multimodal_search(rag, "Was weisst Du über Cyborgs?")
mprint(result)

# 4 | Suche Bild -> Text/Bild
---

In [ ]:
# Bild → Bild: Finde visuell ähnliche Bilder
similar_images = search_similar_images(rag, "./zwei_roboter.png", k=5)
mprint("## 🖼️ Suche Bild → Bild")
mprint("---")
for img in similar_images:
    mprint(f"{img['filename']}: Ähnlichkeit: {img['similarity']}")

In [ ]:
# Bild → Text: Finde Textinformationen zu dem Bild
text_result = search_text_by_image(rag, "./zwei_roboter.png", k=5)
mprint(text_result)